# Genie Bot MVP - Deployment

Deploy minimalista do Genie Bot usando MLflow ResponsesAgent

In [ ]:
# Cell 1: Install dependencies
%pip install -U -qqqq databricks-openai backoff databricks-agents mlflow-skinny[databricks]
dbutils.library.restartPython()

## Configuração

**IMPORTANTE**: Antes de executar, edite o arquivo `genie_bot.py` e substitua `SEU_SPACE_ID_AQUI` pelo Space ID do seu Genie Space

In [ ]:
# Cell 2: Testar localmente
from genie_bot import AGENT

result = AGENT.predict({
    "input": [{
        "role": "user",
        "content": "Quantas verificações tivemos em 2024?"
    }]
})
print(result)

In [ ]:
# Cell 3: Log model
import mlflow
from mlflow.models.resources import DatabricksServingEndpoint, DatabricksFunction

input_example = {
    "input": [{"role": "user", "content": "Olá, quantas verificações em 2024?"}]
}

resources = [
    DatabricksServingEndpoint(endpoint_name="databricks-llama-4-maverick"),
    DatabricksFunction(function_name="hs_franquia.gold_connect_bot.query_genie")
]

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="genie_bot",
        python_model="genie_bot.py",
        input_example=input_example,
        pip_requirements=[
            "databricks-openai",
            "backoff",
        ],
        resources=resources,
    )

print(f"Model logged: {logged_agent_info.model_uri}")

In [ ]:
# Cell 4: Register to UC
mlflow.set_registry_uri("databricks-uc")
UC_MODEL_NAME = "hs_franquia.gold_connect_bot.genie_bot"

uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri,
    name=UC_MODEL_NAME
)

print(f"Model registered: {UC_MODEL_NAME} version {uc_registered_model_info.version}")

In [ ]:
# Cell 5: Deploy
from databricks import agents

deployment_info = agents.deploy(
    UC_MODEL_NAME,
    uc_registered_model_info.version,
    scale_to_zero=True
)

print(f"Deployed! Endpoint: {deployment_info}")

## Próximos Passos

1. Teste o endpoint via AI Playground
2. Teste via API REST
3. Valide as respostas do Genie
4. Se funcionar, adicione features incrementalmente:
   - Access control
   - Conversation history
   - Large result uploads
   - Vector Search para docs